# Imports

In [ ]:
import os
import glob
import gzip
import pickle
import pandas as pd
from ete3 import NCBITaxa
import ncbi_genome_download as ngd

# Create index file

## Taxanomy tree

In [ ]:
ncbi = NCBITaxa()
ncbi.update_taxonomy_database()

## Taxa for fusobacteria

In [3]:
taxon_name = 'fusobacteria'
ebact = ncbi.get_descendant_taxa(taxon_name, return_tree=True)
with open('./taxaindex', 'w') as wr:
    for i in ebact:        
        wr.write("%s\n" % i)

## Download genome from taxa

In [ ]:
ngd.download(assembly_level='complete',
             taxid ='./taxaindex',
             file_format = 'fasta',
             output='out',
             group='bacteria')

# Creae index table

## Read genomes

In [ ]:
def clear(dna):
    out = list()
    for i in range(0,len(dna)):
        if(dna[i] in ['A','C','G','T']):            
            out.append(dna[i])
    return ''.join(out)

In [5]:
files = [x[0] for x in os.walk('./out/refseq/bacteria/')][1:]

genoms = list()

df = pd.read_csv('./assembly_summary.txt',
                 sep='\t',
                 lineterminator='\n') # for retrieve taxaid

for file in files:           
    query = file.split('/')[-1]
    taxid = int(df.loc[df['gbrs_paired_asm'] == query]['taxid'])    
    print(file + " , "+ str(taxid))    
    fna = glob.glob(file+"/*.fna.gz")[0]
    f = gzip.open(fna, 'rb')    
    file_content = str(f.read())
    f.close()     
    parts = file_content.split('>N')                      
    dna =""
    for p in parts:
        if(len(p)>len(dna)):
            dna = p    
    dna = dna.split('\\n')          
    dna = dna[1:]    
    dna = ''.join(dna)        
    dna = clear(dna)
    genoms.append([dna,taxid])    
del(df)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


./out/refseq/bacteria/GCF_000007325.1 , 190304
./out/refseq/bacteria/GCF_000023905.1 , 523794
./out/refseq/bacteria/GCF_000024405.1 , 526218
./out/refseq/bacteria/GCF_000024565.1 , 519441
./out/refseq/bacteria/GCF_000158275.2 , 457405
./out/refseq/bacteria/GCF_000162235.2 , 469604
./out/refseq/bacteria/GCF_000163915.2 , 469602
./out/refseq/bacteria/GCF_000165505.1 , 572544
./out/refseq/bacteria/GCF_000400875.1 , 469607
./out/refseq/bacteria/GCF_000973085.1 , 187101
./out/refseq/bacteria/GCF_001274535.1 , 712357
./out/refseq/bacteria/GCF_001296125.1 , 1307427
./out/refseq/bacteria/GCF_001296165.1 , 1307428
./out/refseq/bacteria/GCF_001455085.1 , 1307443
./out/refseq/bacteria/GCF_001455105.1 , 1307444
./out/refseq/bacteria/GCF_001455145.1 , 1307442
./out/refseq/bacteria/GCF_001553645.1 , 1785996
./out/refseq/bacteria/GCF_002240055.1 , 712368
./out/refseq/bacteria/GCF_002763595.1 , 2663009
./out/refseq/bacteria/GCF_002763625.1 , 2663009
./out/refseq/bacteria/GCF_002763695.1 , 2663009
./ou

In [9]:
def reverse(seq): 
    rev = seq[::-1]
    rev = rev.replace("A","X")
    rev = rev.replace("T","A")
    rev = rev.replace("X","T")
    rev = rev.replace("C","X")
    rev = rev.replace("G","C")
    rev = rev.replace("X","G")
    return rev

In [10]:
# parameter
M=7 # minimizer parameter
kmer=31


index = {}
ancesotr = {}
for gene in genoms:    
    seq = gene[0]
    taxid = gene[1]
    print(taxid)    
    # kmer
    L = len(seq)
    for i in range(0,L-kmer+1):
        sub_f=seq[i:i+kmer]        
        # sub_r = rev[L-kmer-i:L-i]
        sub_r = reverse(sub_f)
        mini = "ZZZZZZZZZZZZZ"
        # minimizer
        for j in range(0, kmer-M+1):
                sub2=sub_f[j:j+M]
                if sub2 < mini:
                    mini=sub2
                sub2=sub_r[j:j+M]
                if sub2 < mini:
                    mini=sub2
        h = sub_f
        if mini in index:            
            if h not in index[mini]: # add minimizer to index table               
                index[mini][h] = taxid
            elif(taxid != index[mini][h]): 
                s = str([taxid,index[mini][h]])
                if(s in ancesotr):
                    index[mini][h] = ancesotr[s]
                else:                                       
                    index[mini][h] = int(ebact.get_common_ancestor(str(index[mini][h]),str(taxid)).name)
                    ancesotr[s] = index[mini][h]
        else:             
            index[mini]={h:taxid}

190304
523794
526218
519441
457405
469604
469602
572544
469607
187101
712357
1307427
1307428
1307443
1307444
1307442
1785996
712368
2663009
2663009
2663009
2663009
2663009
2663009
2663009
2663009
2663009
469616
469618
469615
525283
143388
554406
469617


In [5]:
with open("index.p", "wb") as file: # save index table
    pickle.dump(index, file)

# Load index table and predict

In [6]:
with open("index.p", "rb") as file: # save index table
    index = pickle.load(file)

## Read metagenomics

In [23]:
file = open('metagenomic.txt','r')
reads = file.read()
file.close()
reads = reads.split('\n')

In [48]:
def kraken(seq):
    score={}
    kmer=31
    M=7
    rev=reverse(seq)
    # kmer
    L = len(seq)
    for i in range(0,L- kmer +1):
        sub_f=seq[i:i+kmer]
        #sub_r=rev[L-kmer-i:L-i]
        sub_r = reverse(sub_f)
        mini = "ZZZZZZZZZZZZZ"
        # minimizer
        for j in range(0, kmer-M+1):
            sub2=sub_f[j:j+M]
            if sub2 < mini:
                mini = sub2                
            sub2 = sub_r[j:j+M]
            if sub2 < mini:
                mini=sub2        
        if(mini in index):
            if(sub_f in index[mini]):
                id = index[mini][sub_f]                
            else :
                continue
        else :
            continue
        if(id in score):
            score[id] = score[id] + 1
        else:
            score[id] = 1
    # select max score
    maxs = 0
    answer = 0
    for item in score:
        s = 0
        node = ebact.search_nodes(name=str(item))[0]
        ancestors = node.get_ancestors()
        for anc in ancestors: 
            if(anc in score):
                s += score[anc]
        if(s > maxs):
            maxs = s
            answer = item 
    return answer

In [ ]:
acc = 0
for read in reads:
    read = read.split(',')
    taxid =read[1]
    read = read[0]    
    extimeted = kraken(read)
    if(extimeted == taxid):
        acc += 1
acc = (acc / len(reads)) * 100
print('accuracy : ' + str(acc) + "%")